<a href="https://colab.research.google.com/github/ChapelFob80930/AI-Generated-Text-Detector/blob/main/AITextTest2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [68]:
import numpy as np
import pandas as pd

train_prompts = pd.read_csv("https://raw.githubusercontent.com/ChapelFob80930/AI-Generated-Text-Detector/refs/heads/main/train_prompts.csv")
print(train_prompts.head())
print("\n")
train_essays = pd.read_csv("https://raw.githubusercontent.com/ChapelFob80930/AI-Generated-Text-Detector/refs/heads/main/train_essays.csv")
print(train_essays.head())
print("\n")
test_essays = pd.read_csv("https://raw.githubusercontent.com/ChapelFob80930/AI-Generated-Text-Detector/refs/heads/main/test_essays.csv")
print(test_essays.head())
# ai_human_dataset = pd.read_csv("/content/AI_Human.csv")
# print(ai_human_dataset.head())

   prompt_id                       prompt_name  \
0          0                   Car-free cities   
1          1  Does the electoral college work?   

                                        instructions  \
0  Write an explanatory essay to inform fellow ci...   
1  Write a letter to your state senator in which ...   

                                         source_text  
0  # In German Suburb, Life Goes On Without Cars ...  
1  # What Is the Electoral College? by the Office...  


         id  prompt_id                                               text  \
0  0059830c          0  Cars. Cars have been around since they became ...   
1  005db917          0  Transportation is a large necessity in most co...   
2  008f63e3          0  "America's love affair with it's vehicles seem...   
3  00940276          0  How often do you ride in a car? Do you drive a...   
4  00c39458          0  Cars are a wonderful thing. They are perhaps o...   

   generated  
0          0  
1          0  
2    

In [69]:

import nltk
nltk.download()


NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> all


       | 
       | Downloading package abc to /root/nltk_data...
       |   Package abc is already up-to-date!
       | Downloading package alpino to /root/nltk_data...
       |   Package alpino is already up-to-date!
       | Downloading package averaged_perceptron_tagger to
       |     /root/nltk_data...
       |   Package averaged_perceptron_tagger is already up-to-date!
       | Downloading package averaged_perceptron_tagger_eng to
       |     /root/nltk_data...
       |   Package averaged_perceptron_tagger_eng is already up-to-
       |       date!
       | Downloading package averaged_perceptron_tagger_ru to
       |     /root/nltk_data...
       |   Package averaged_perceptron_tagger_ru is already up-to-
       |       date!
       | Downloading package averaged_perceptron_tagger_rus to
       |     /root/nltk_data...
       |   Package averaged_perceptron_tagger_rus is already up-to-
       |       date!
       | Downloading package basque_grammars to /root/nltk_data...
     


---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> q


True

In [70]:
import spacy
nlp = spacy.load('en_core_web_sm')

def lemmatize_text(text):
  doc = nlp(" ".join(text))
  return [token.lemma_ for token in doc]

In [71]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

stop_words = set(stopwords.words('english'))

def _remove_stopwords_(texts):
  return [text for text in texts if text.isalnum() and text not in stop_words]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [72]:
from nltk.tokenize import word_tokenize
nltk.download('punkt')

def preprocess_text(series):
    def process(text):
        tokens = word_tokenize(text)
        tokens = _remove_stopwords_(tokens)
        tokens = lemmatize_text(tokens)
        return " ".join(tokens)
    return series.apply(process)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [73]:
train_essays["preprocessedText"]=preprocess_text(train_essays['text'])

In [74]:
print(train_essays.head())

         id  prompt_id                                               text  \
0  0059830c          0  Cars. Cars have been around since they became ...   
1  005db917          0  Transportation is a large necessity in most co...   
2  008f63e3          0  "America's love affair with it's vehicles seem...   
3  00940276          0  How often do you ride in a car? Do you drive a...   
4  00c39458          0  Cars are a wonderful thing. They are perhaps o...   

   generated                                   preprocessedText  
0          0  car Cars around since become famous 1900s Henr...  
1          0  transportation large necessity country worldwi...  
2          0  America love affair vehicle seem cool say Elis...  
3          0  how often ride car do drive one motor vehicle ...  
4          0  car wonderful thing they perhaps one world gre...  


In [75]:
from gensim.models import Word2Vec

sentences = train_essays['preprocessedText'].apply(lambda x: x.split())

word2vec_model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)

word_vector = word2vec_model.wv['car']
print("Vector for 'car':", word_vector)

# train_essays['word2vecVector'] = sentences.apply(
#     lambda x: sum([word2vec_model.wv[word] for word in x if word in word2vec_model.wv]) / len(x)
# )

def calculate_average_word_vector(words,model,vector_size=100):
    word_vectors = [word2vec_model.wv[word] for word in words if word in word2vec_model.wv]
    if word_vectors:
        return np.mean(word_vectors, axis=0)
    else:
        return np.zeros(vector_size)

train_essays['word2vecVector'] = train_essays['preprocessedText'].apply(lambda x: calculate_average_word_vector(x.split(),word2vec_model))


Vector for 'car': [ 1.3983139  -0.19952865 -0.8975377   1.0157179   0.1005862  -1.8195223
  2.36432    -0.35210788 -1.5325035   1.0050671   0.32618183  2.19435
  0.9945756   1.4188082  -0.76280695 -1.1882021   0.34617773 -1.201802
 -1.5399814  -2.1809256   2.4871314   0.07184775 -0.01507976  0.0372612
 -0.43341225 -1.1171083  -1.1511848  -0.41702014 -0.90715325 -0.5025374
 -1.0196035   0.9593533   0.2742427   0.33625245  0.778704    0.14398967
  0.03318926 -1.2424289   0.72138244 -2.9377427  -0.14957233  0.1440818
 -0.92040324  0.06196193 -0.0184778   0.16242434 -0.7733787   1.2210757
  2.6344109   1.4838057   0.25009915  1.6349393   0.07421927  0.26863998
  1.3208207  -0.9120307   0.6753075  -0.6227707  -1.5856438  -0.13071328
  0.5721468  -0.09824565 -0.47199765 -1.8865796  -1.0702983   1.4756329
  0.11871076  0.13812591 -0.46286443  0.4483024   0.91647154  0.02716387
 -0.42406774 -2.0444653   1.154392   -0.7607485   2.7832      0.35853878
 -1.646011   -0.07624491  0.03392937  0.6824

In [76]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

tagged_data = [TaggedDocument(words=text.split(), tags=[str(i)]) for i, text in enumerate(train_essays['preprocessedText'])]

doc2vec_model = Doc2Vec(tagged_data, vector_size=100, window=5, min_count=1, workers=4)

train_essays['doc2vecVector'] = train_essays.index.map(lambda x: doc2vec_model.dv[str(x)])


In [77]:
print(train_essays.head())

         id  prompt_id                                               text  \
0  0059830c          0  Cars. Cars have been around since they became ...   
1  005db917          0  Transportation is a large necessity in most co...   
2  008f63e3          0  "America's love affair with it's vehicles seem...   
3  00940276          0  How often do you ride in a car? Do you drive a...   
4  00c39458          0  Cars are a wonderful thing. They are perhaps o...   

   generated                                   preprocessedText  \
0          0  car Cars around since become famous 1900s Henr...   
1          0  transportation large necessity country worldwi...   
2          0  America love affair vehicle seem cool say Elis...   
3          0  how often ride car do drive one motor vehicle ...   
4          0  car wonderful thing they perhaps one world gre...   

                                      word2vecVector  \
0  [0.13709943, 0.04006377, -0.54692876, -0.22202...   
1  [0.20429766, 0.0841

In [78]:
from sklearn.model_selection import train_test_split

y = train_essays['generated']

train_index, test_index = train_test_split(train_essays.index, test_size=0.2, random_state=42)

In [79]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(max_features=1000)
X_tfidf = tfidf_vectorizer.fit_transform(train_essays['preprocessedText'])

X_train_tfidf, X_test_tfidf = X_tfidf[train_index], X_tfidf[test_index]
y_train, y_test = y[train_index], y[test_index]


In [80]:
train_essays['word2vecVector'] = train_essays['word2vecVector'].apply(lambda x: np.array(x))

X_word2vec = np.stack(train_essays['word2vecVector'].values)

X_train_word2vec, X_test_word2vec = X_word2vec[train_index], X_word2vec[test_index]

print(X_train_word2vec.shape)
print(X_test_word2vec.shape)
print(X_train_word2vec)

(1102, 100)
(276, 100)
[[ 0.19556035  0.01397521 -0.53134847 ... -0.39088723  0.29515126
  -0.08978265]
 [ 0.05017787  0.20533411 -0.42644942 ... -0.13210651  0.16349831
   0.10699563]
 [-0.7282712   0.5616144   0.5089235  ... -0.46966726  0.117281
   0.10630482]
 ...
 [ 0.05311797  0.07651582 -0.3217285  ... -0.57073444  0.30312976
  -0.25136626]
 [-0.89038473  0.55204153  0.6662016  ... -0.33236763  0.03629508
   0.26991636]
 [ 0.00541546  0.37701634 -0.37251607 ... -0.12648192  0.08343007
  -0.08109234]]


In [83]:
train_essays['doc2vecVector'] = train_essays['doc2vecVector'].apply(lambda x: np.array(x))
X_doc2vec = np.stack(train_essays['doc2vecVector'].values)
X_train_doc2vec, X_test_doc2vec = train_essays['doc2vecVector'][train_index], train_essays['doc2vecVector'][test_index]
print(X_train_doc2vec.shape)
print(X_test_doc2vec.shape)
print(X_train_doc2vec)

Data type of vectors: <class 'numpy.ndarray'>
Vector shape: (100,)
(1102,)
(276,)
1209    [-0.07081423, -0.27147925, -0.36270842, -0.278...
309     [-0.2787647, -0.32687306, -0.60859895, -0.1942...
490     [-0.6622444, 0.045042988, 0.39596534, -0.11355...
1125    [-0.2729457, 0.37365356, -0.3988073, 0.2193811...
535     [-0.14463139, 0.33074507, -0.51011497, 0.20518...
                              ...                        
1095    [-0.23337848, -0.3155109, -0.19650547, 0.04177...
1130    [-0.19660729, 0.08925833, -0.11477024, 0.10518...
1294    [-0.11060697, -0.28280225, -0.41419506, -0.215...
860     [-0.64594334, 0.18687847, 0.21592201, -0.00725...
1126    [-0.21063812, -0.077271186, -0.3586669, -0.441...
Name: doc2vecVector, Length: 1102, dtype: object


In [82]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Function to train and evaluate a model
def evaluate_model(X_train, X_test, y_train, y_test):
    svm_model = SVC(class_weight='balanced')
    svm_model.fit(X_train, y_train)
    y_pred = svm_model.predict(X_test)
    return {
        "Accuracy": accuracy_score(y_test, y_pred),
        "Precision": precision_score(y_test, y_pred),
        "Recall": recall_score(y_test, y_pred),
        "F1-Score": f1_score(y_test, y_pred)
    }

# Evaluate TF-IDF
results_tfidf = evaluate_model(X_train_tfidf, X_test_tfidf, y_train, y_test)
print("TF-IDF Results:", results_tfidf)

# Evaluate Word2Vec
results_word2vec = evaluate_model(X_train_word2vec, X_test_word2vec, y_train, y_test)
print("Word2Vec Results:", results_word2vec)

# Evaluate Doc2Vec
results_doc2vec = evaluate_model(X_train_doc2vec, X_test_doc2vec, y_train, y_test)
print("Doc2Vec Results:", results_doc2vec)


TF-IDF Results: {'Accuracy': 0.9963768115942029, 'Precision': 0.0, 'Recall': 0.0, 'F1-Score': 0.0}
Word2Vec Results: {'Accuracy': 0.9891304347826086, 'Precision': 0.0, 'Recall': 0.0, 'F1-Score': 0.0}


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


ValueError: setting an array element with a sequence.